sbpy.data.Phys Example Notebook
=================================

[sbpy.data.Phys](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Phys.html#sbpy.data.Phys) provides functionality to query and store small-body physical properties.

Querying Physical Properties from JPL SBDB
--------------------------------------------

Query the physical properties for a few asteroids:

In [1]:
from sbpy.data import Phys

phys = Phys.from_sbdb(['433', 'Itokawa', '12893'])
print(phys.column_names)

<TableColumns names=('targetname','H','H_sig','H_ref','H_note','G','G_sig','G_ref','G_note','diameter','diameter_sig','diameter_ref','diameter_note','extent','extent_sig','extent_ref','extent_note','GM','GM_sig','GM_ref','GM_note','density','density_sig','density_ref','density_note','rot_per','rot_per_sig','rot_per_ref','rot_per_note','albedo','albedo_sig','albedo_ref','albedo_note','BV','BV_sig','BV_ref','BV_note','UB','UB_sig','UB_ref','UB_note','spec_T','spec_T_sig','spec_T_ref','spec_T_note','spec_B','spec_B_sig','spec_B_ref','spec_B_note')>


We can calculate their volumes assuming spherical shapes:

In [2]:
import numpy as np
list(zip(phys['targetname'], 4/3*np.pi*phys['diameter']**3))

[('433 Eros (1898 DQ)', <Quantity 20003.90902611 km3>),
 ('25143 Itokawa (1998 SF36)', <Quantity 0.15053255 km3>),
 ('12893 Mommert (1998 QS55)', <Quantity 593.74735751 km3>)]

Please keep in mind that physical properties information provided by SBDB is incomplete and scarce. Missing values are replaced with `nan`.

Querying Molecular Data from `astroquery.jplspec`
==========================

`sbpy.data.Phys` also contains a function to query molecular data that
might be useful for various calculations such as production rate calculations.
`sbpy.data.Phys.from_jplspec` queries the [JPL Molecular Spectroscopy Catalog](https://spec.jpl.nasa.gov/home.html) molecular properties, and stores the
data in a `sbpy.data.Phys` object, offering the same functionality as all the
other `sbpy.data` functions, including the use of `~astropy.units`. `sbpy.data.Phys.from_jplspec` also
calculates the partition function at the desired temperature using log-space interpolation. For
a briefing of how this is done look at the sbpy jplspec notebook: [jplspec notebook](jplspec.ipynb). The results
from `sbpy.data.Phys.from_jplspec` include the following data:

- Transition frequency
- Temperature
- Integrated line intensity at 300 K
- Partition function at 300 K
- Partition function at designated temperature
- Upper state degeneracy
- Upper level energy in Joules
- Lower level energy in Joules
- Degrees of freedom

For the names of these fields and their alternatives, see [here](https://sbpy.readthedocs.io/en/latest/sbpy/data/fieldnames.html#id1)

In [5]:
import astropy.units as u

In [8]:
temp_estimate = 47. * u.K # kinetic temperature

mol_tag = 28001 # JPLSpec unique identifier

transition_freq = (345.7 * u.GHz).to('MHz') # Transition frequency

mol_data = Phys.from_jplspec(temp_estimate, transition_freq, mol_tag) # build Phys object

Once the phys object has been created, one can access all the information given by astroquery.jplspec as well as the partition function at the desired temperature:

In [9]:
print(mol_data['elo_j']) # print energy of lower level
print(mol_data['degfr']) # print degrees of freedom
print(mol_data['partfn']) # print interpolated partition function at desired temp

[2.29136526e-22] J
degfreedom
----------
         2
      partfn     
-----------------
17.05551276908476


Regular expressions can also be used as molecule identifiers since astroquery.jplspec provides the functionality. It is important to understand regular expressions and how to use them in a way that gets the user exactly what they want. If the user is unfamiliar with regular expressions, using the JPL Spectral catalog unique identifier is the best course of action.

Hint: in regular expressions, putting something between '^' and '\$' will match the literal text in between. This is useful to remember in order to avoid matching unnecessary terms. i.e. using 'co' will match 'co', 'co2', 'hco' so you would want to input '^co$' to avoid such nuances

In [10]:
mol_tag = '^CO$'

mol_data = Phys.from_jplspec(temp_estimate, transition_freq, mol_tag) # build Phys object

print(mol_data['elo_j']) # print energy of lower level
print(mol_data['degfr']) # print degrees of freedom
print(mol_data['partfn']) # print interpolated partition function at desired temp

[2.29136526e-22] J
degfreedom
----------
         2
      partfn     
-----------------
17.05551276908476
